In [ ]:
import hyperspy.api as hs
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import os

from helper_functions import elementlines, nearestlines, theoretical_energy

In [ ]:
%matplotlib qt

In [ ]:
##### SEM Apreo March 2023 #####
path = '../../Masteroppgave/2023-03-08_EDS-Apreo/exports/'
aztec_file = '../../Masteroppgave/2023-03-08_EDS-Apreo/AZtec k-factors and quantification.xlsx'
nans = np.array([0,3,6,9,10,11,12,15,16,19,22,25,28,31,34,37,40,43,46,49,52,55,56,57,60])+1
df = pd.read_excel(aztec_file, sheet_name=4, skiprows=nans)

In [ ]:
df['Name'] = df['File']
df['Name'] = df['Name'].str.replace('_', ' ')
df['Name'] = df['Name'].str.replace('GaAs', '(a)')
df['Name'] = df['Name'].str.replace('GaSb', '(b)')
df['Name'] = df['Name'].str.replace('processTime', 'PT')
df['Name'] = df['Name'].str.replace('scratched', 'scr')
df['Line'] = df['Element'] + '_' + df['Line series'] + 'a'
df['Line'] = df['Line'].str.replace(' ', '')
df['i'] = 0.0
df['i/i_sum'] = 0.0


df.head(3)

In [ ]:
def calculate_intensities(filename):

        # check if filename exists in path
    if filename + '.emsa' not in os.listdir(path):
        print(filename + '.emsa', 'not found in path')
        return
    
    df_s = df[df['File'] == filename]
    df_s = df_s.sort_values(by=['Element']) # alphabetically, as HS

    noise_slice = 0.65 # keV

    # make the spectrum and the model
    s = hs.load(path + filename + '.emsa', signal_type='EDS_TEM')
    elements = df_s['Element'].values
    s = s.isig[noise_slice:] # noise peak slice
    if s.metadata.Acquisition_instrument.TEM.beam_energy < 20:
        s = s.isig[:s.metadata.Acquisition_instrument.TEM.beam_energy]
    s.add_elements(elements)
    s.add_lines()
    m = s.create_model()
    m.fit()
    m.fit_background()

    lines = df_s['Line'].values

    line_intensities = []
    for l in lines:
        line_intensities.append(int(m[l].A.value))

    relative_intensities = []
    for line_intens in line_intensities:
        relative_intensities.append(round(line_intens/sum(line_intensities), 3))

    df_s['i'] = line_intensities
    df_s['i/i_sum'] = relative_intensities


    df[df['File'] == filename] = df_s

calculate_intensities('GaAs_30kV_50pA')

In [ ]:
for filename in df['File'].unique():
    calculate_intensities(filename)
df

In [ ]:
df.plot(x='Name', y='i/i_sum', kind='bar', figsize=(20,10), title='Relative intensities of lines with no correction')